In [1]:
import pathlib

import pymeshlab
from pymeshlab import AbsoluteValue, MeshSet

from cfdmod.api.geometry.STL import export_stl, read_stl
from lnas import LnasGeometry
from lnas.transformations import TransformationsMatrix

import numpy as np


def apply_remeshing(element_size: float, mesh_path: pathlib.Path, output_path: pathlib.Path):
    ms: MeshSet = pymeshlab.MeshSet()
    ms.load_new_mesh(str(mesh_path))
    ms.meshing_isotropic_explicit_remeshing(
        iterations=15, targetlen=AbsoluteValue(element_size), selectedonly=False, featuredeg=180
    )
    ms.save_current_mesh(str(output_path), binary=True)


def correct_normals(mesh_path: pathlib.Path, output_path: pathlib.Path):
    ms: MeshSet = pymeshlab.MeshSet()
    ms.load_new_mesh(str(mesh_path))
    ms.compute_selection_by_condition_per_face(condselect="fnz<0")
    ms.meshing_invert_face_orientation(onlyselected=True)
    ms.save_current_mesh(str(output_path), binary=True)


def translate_mesh(
    translation: tuple[float, float, float],
    mesh_path: pathlib.Path,
    output_path: pathlib.Path,
):
    triangles, normals = read_stl(mesh_path)
    triangles[:, :, 0] += translation[0]
    triangles[:, :, 1] += translation[1]
    triangles[:, :, 2] += translation[2]
    export_stl(output_path, triangles, normals)


def rotate_mesh(
    rotation: tuple[float, float, float],
    fixed_point: tuple[float, float, float],
    mesh_path: pathlib.Path,
    output_path: pathlib.Path,
):
    triangles, normals = read_stl(mesh_path)
    T = TransformationsMatrix(angle=rotation, fixed_point=fixed_point)
    for idx, points in enumerate(triangles):
        points = T.apply_points(points)
        triangles[idx, :, :] = points

    export_stl(output_path, triangles, normals)

In [2]:
import numpy as np


def slice_surface(
    plane_normal: tuple[float, float, float],
    plane_origin: tuple[float, float, float],
    mesh_path: pathlib.Path,
):
    distance = np.dot(plane_normal, plane_origin) / np.linalg.norm(plane_normal)
    ms: MeshSet = pymeshlab.MeshSet()
    ms.load_new_mesh(str(mesh_path.absolute()))
    ms.generate_polyline_from_planar_section(
        planeaxis="Custom Axis",
        customaxis=plane_normal,
        planeoffset=distance,
        relativeto="Origin",  # 'Bounding box center' 'Bounding box min'
        splitsurfacewithsection=True,
    )

    return ms

In [3]:
def get_mesh_coordinates(mesh_path: pathlib.Path) -> tuple[np.ndarray, np.ndarray]:
    triangles, normals = read_stl(mesh_path)

    return triangles, normals

In [4]:
def get_directory_content(dir_root: pathlib.Path):
    dir_tree = {}
    dir_tree["_files"] = []
    for item in dir_root.iterdir():
        if item.is_dir():
            dir_tree[item] = get_directory_content(item)
        elif item.is_file():
            dir_tree["_files"].append(item)
    return dir_tree

## Example usage

### Slice surface

In [19]:
folder_path = pathlib.Path("/mnt/disk01/prd-eng/consulting/033-GolgiCajamar/default/STLs/terrain/")

(x, y) = (0, 0)
plane_normal = [x / 1300, y / 1300, 0]
plane_origin = [x, y, 0]

dir_tree = get_directory_content(folder_path)
mesh_list = dir_tree["_files"]
for mesh_name in mesh_list:
    ms = slice_surface(plane_normal, plane_origin, mesh_path=mesh_name)
    for index in [2, 3]:
        output_path = mesh_name.parent / f"{mesh_name.name}-{index-2}.stl"
        ms.set_current_mesh(index)
        ms.save_current_mesh(str(output_path.absolute()))

### Cut square from surface

In [4]:
for i in [2]:
    folder_path = pathlib.Path(
        f"/mnt/disk01/prd-eng/consulting/035-GLPBandeirantesGalpoesCeD/default/STLs/"
    )

    radius = 1300
    center = (0, 0)

    dir_tree = get_directory_content(folder_path)
    mesh_list = dir_tree["_files"]

    for mesh_name in mesh_list:
        output_path = mesh_name.parent / f"{mesh_name.name}-cut.stl"
        c = center
        for x, y in (
            (c[0], c[1] - radius),
            (c[0], c[1] + radius),
            (c[0] - radius, c[1]),
            (c[0] + radius, c[1]),
        ):
            plane_normal = [x / radius, y / radius, 0]
            plane_origin = [x, y, 0]
            ms = slice_surface(plane_normal, plane_origin, mesh_path=mesh_name)
            ms.set_current_mesh(3)
            ms.save_current_mesh(str(output_path.absolute()))
            mesh_name = output_path

### Remesh

In [36]:
folder_path = pathlib.Path(
    "/home/ubuntu/repos/drives/consulting/Consulting/026 - Prologis Cajamar 4/Arquivos Trabalhados/G100/16-01-2024/lanternim/"
)
output_path = pathlib.Path(
    "/home/ubuntu/repos/drives/consulting/Consulting/026 - Prologis Cajamar 4/Arquivos Trabalhados/G100/16-01-2024/lanternim/"
)

dir_tree = get_directory_content(folder_path)
mesh_list = dir_tree["_files"]
for mesh_name in mesh_list:
    file_name = mesh_name.name
    apply_remeshing(
        element_size=0.5,
        mesh_path=mesh_name,
        output_path=output_path / file_name,
    )

### Translate

In [5]:
folder_path = pathlib.Path(
    "/home/ubuntu/repos/drives/consulting/Consulting/030 - Monto Pirelli Campinas/Arquivos Trabalhados/G100/STLs/final/"
)
output_path = pathlib.Path(
    "/mnt/disk01/prd-eng/consulting/030_MontoPirelliCampinas/STLs/STLs_dirs/G100/"
)
translation = [-147.102412, 86.014694, 628.2]


dir_tree = get_directory_content(folder_path)
mesh_list = dir_tree["_files"]
for mesh_name in mesh_list:
    file_name = mesh_name.name
    translate_mesh(
        translation=translation,
        mesh_path=mesh_name,
        output_path=output_path / file_name,
    )

### Rotate

In [ ]:
folder_path = pathlib.Path(
    "/mnt/disk01/prd-eng/consulting/033-GolgiCajamar/direcoes_adicionais/01/004/constant/triSurface/"
)
output_path = pathlib.Path(
    "/mnt/disk01/prd-eng/consulting/033-GolgiCajamar/direcoes_adicionais/01/004/constant/triSurface/out/"
)
rotation = np.array([0, 0, -0.0659734457253856])
fixed_point = np.array([0, 0, 0])

dir_tree = get_directory_content(folder_path)
mesh_list = dir_tree["_files"]

for mesh_name in mesh_list:
    file_name = mesh_name.name
    print(output_path / file_name)
    rotate_mesh(
        rotation=rotation,
        fixed_point=fixed_point,
        mesh_path=mesh_name,
        output_path=output_path / file_name,
    )

### Point normals downward

In [12]:
direction = "315"
folder_path = pathlib.Path(
    f"/mnt/disk01/prd-eng/consulting/030_MontoPirelliCampinas/artifacts/STLs_files/{direction}/"
)
output_path = pathlib.Path(
    f"/mnt/disk01/prd-eng/consulting/030_MontoPirelliCampinas/artifacts/STLs_files/{direction}"
)

dir_tree = get_directory_content(folder_path)
mesh_list = dir_tree["_files"]
for mesh_name in mesh_list:
    file_name = mesh_name.name
    correct_normals(mesh_path=mesh_name, output_path=output_path / file_name)

### Extract stl vertices coordinates and export to csv

In [12]:
import pandas as pd

stl_path = pathlib.Path(
    f"/mnt/disk01/prd-eng/consulting/036-FGEngenhariaPostoNO/setup/STLs/top_offset.stl"
)
output_path = pathlib.Path(
    f"/mnt/disk01/prd-eng/consulting/036-FGEngenhariaPostoNO/simulation_data/artifacts/csvs/"
)

triangles, normals = get_mesh_coordinates(stl_path)
centers = np.average(triangles, axis=1)

columns = ["x", "y", "z"]
df = pd.DataFrame(data=centers, columns=columns)
df.to_csv(output_path / "top_offset.csv", index=None)

### Finds the crossing z points on an STL(lnas) corresponding to a list of xy coordinates

In [15]:
import pathlib
from lnas import LnasFormat
import numpy as np
import pandas as pd
import trimesh

root_path = pathlib.Path(f"/mnt/disk01/prd-eng/consulting/036-FGEngenhariaPostoNO/")
lnas_path = root_path / "simulation_data/artifacts/lnas/bottom_offset.lnas"
points_path = root_path / "simulation_data/artifacts/csvs/top_2.csv"

lnas_fmt = LnasFormat.from_file(pathlib.Path(lnas_path))
lnas_geo = lnas_fmt.geometry

tri_up = lnas_geo.normals[:, 2] > 0.5
tri_down = lnas_geo.normals[:, 2] < -0.5
tri_up_idxs = np.arange(len(tri_up))[tri_up]
tri_down_idxs = np.arange(len(tri_down))[tri_down]

lnas_up = lnas_fmt.filter_triangles(tri_up)
lnas_down = lnas_fmt.filter_triangles(tri_down)

points = pd.read_csv(points_path, sep=",")
arr_points = np.array([points[d].to_numpy() for d in ["x", "y", "z"]], dtype=np.float32)
arr_points = arr_points.swapaxes(0, 1)

mesh_up = trimesh.Trimesh(
    vertices=lnas_up.geometry.vertices.copy(), faces=lnas_up.geometry.triangles.copy()
)
mesh_down = trimesh.Trimesh(
    vertices=lnas_down.geometry.vertices.copy(), faces=lnas_down.geometry.triangles.copy()
)


def check_intersection(mesh, ray_origin, ray_direction):
    # Create a ray from start_point to end_point
    ray = trimesh.ray.ray_triangle.RayMeshIntersector(mesh)
    intersects, index_tri, index_ray = ray.intersects_location([ray_origin], [ray_direction])

    if intersects.any():
        # If any intersection is found, return the indices of triangles intersected
        return intersects, index_tri, index_ray
    raise ValueError("not interecpts")


normals = []

for idx, p in enumerate(arr_points):
    mesh_use = mesh_up
    if p[2] == 10:
        mesh_use = mesh_down
        normals.append([0, 0, -1])
    else:
        normals.append([0, 0, 1])
    ray_origin = [p[0], p[1], 0]
    ray_direction = [0, 0, 1]

    ints, tri_int, index_ray = check_intersection(mesh_use, ray_origin, ray_direction)

    arr_points[idx, 2] = ints[0, 2]

normals = np.array(normals, dtype=np.float32)

df_save = points.copy()
df_save["x"] = arr_points[:, 0]
df_save["y"] = arr_points[:, 1]
df_save["z"] = arr_points[:, 2]
df_save["nx"] = normals[:, 0]
df_save["ny"] = normals[:, 1]
df_save["nz"] = normals[:, 2]


df_save.to_csv(root_path / "simulation_data/artifacts/csvs/bottom_offset.csv", index=False)